01 - Generate JSON from Post-Inclusion/Exclusion CSV
- Enter path to CSV which has successful articles in the index column
- Enter path to JSON which has the articles with labeled sections

In [1]:
csv_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/ocr/keyword_matching_labeled_text/text_evaluations/3-5_inclusion_exclusion_results/unfiltered_results.csv"
json_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/labeled_text/case_labeled_sections.json"

In [2]:
from calvin_utils.gpt_sys_review.json_utils import FilterPapers

# Initialize and run the FilterPapers class
filter_papers = FilterPapers(csv_path=csv_path, json_path=json_path)
filter_papers.run()

/Users/cu135/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


02 - Prepare Data Extraction Questions
- Dev Note: This is a repeat class and should be inherited from a .py

In [3]:
from calvin_utils.gpt_sys_review.gpt_utils import QuestionTemplate

question_template = QuestionTemplate()
question_template.data_extraction_questions()

Here are example data extraction questions:
{
    "Does the patient(s) represent(s) the whole experience of the investigator (center) or is the selection method unclear to the extent that other patients with similar presentation may not have been reported? (Good/Bad/Unclear)": "representative_case_quality",
    "Was patient\u2019s causal exposure clearly described? (Good/Bad/Unclear)": "causality_quality",
    "Were diagnostic tests or assessment methods and the results clearly described (amnesia tests)? (Good/Bad/Unclear)": "phenotyping_quality",
    "Were other alternative causes that may explain the observation (amnesia) ruled out? (Good/Bad/Unclear)": "workup_quality",
    "Were patient\u2019s demographics, medical history, comobidities clearly described? (Good/Bad/Unclear)": "clinical_covariates_quality",
    "Were patient\u2019s symptoms, interventions, and clinical outcomes clearly presented as a timeline? (Good/Bad/Unclear)": "history_quality",
    "Was the lesion image taken a

Copy the Template Dict from Above and Fill it Out As per the Example

In [2]:
question = {
    "Do you think this is reasonably free of selection bias? For example, a case series is free of bias. (Y/N).": "selection_bias",
    "Do you think the history of presenting illness was reasonably described? For example, describing a patient's symptoms and circumstances of presentation are acceptable. (Y/N)": "hpi_history",
    "Prioritizing implicit information, do you think diagnostic diagnostic tests relevant to the diagnosis had been completed to consolidate the diagnosis? Specific details are not necessary. For example, vascular imaging in a stroke is a reasonable workup. (Y/N).": "diagnostic_workup",
    "Do you think the symptom is reasonably attributable to the lesion? For example, a stroke and subsequent amnesia is highly attributable. If the word acute is used, this is a yes. (Y/N)": "diagnostic_accuracy",
    "Do you think the extended patient history was well described? For example, diabetes in pas medical history, family history of strokes, or note of medications. (Y/N)": "associated_history",
    "Considering both explicit mentions and contextual clues like ordered presnetaiton of history to symptoms to workup, can you infer whether the history was presented as a timeline? For example, a description of acute neurological deficit to imaging findings to symptoms is acceptable. (Y/N)": "temporality_of_history",
    "Prioritizing implicit contextual cues, can you infer if the neuroimaging may have been taken around the time of symptom or lesion onset? If neuroimaging is mentioned in close proximity to the description of symptoms, assume they are temporally related. (Y/N)": "lesion_related-images",
    "Do you think the case has sufficient details to allow another doctor to make their own inferences? Specifics are not necessary so long as the critical information is present. For example, if the history and pertinent diagnostic information provided, this is acceptable. (Y/N)": "replicability",
    "Is the case report suitable for lesion network mapping? For example, a case describing symptoms and related neuroimaging related to symptoms is sufficient. (Y/N)": "overall_appraisal"
}

03 - Extract Data with GPT


In [3]:
# Define file paths
api_key_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/openai_key.txt"

# Define the keys you want to consider (exclude 'References')
keys_to_consider = ["case_report"]  # Add or remove keys as per your requirement

# Define the type of article and questions
article_type = "research"

In [4]:
from calvin_utils.gpt_sys_review.gpt_utils import OpenAIChatEvaluator

In [5]:
test_mode=False

In [6]:
evaluator = OpenAIChatEvaluator(api_key_path=api_key_path,
                                json_file_path=json_path, 
                                keys_to_consider=keys_to_consider,
                                question_type=article_type,
                                question=question,
                                test_mode=test_mode,
                                model_choice="gpt4")
answers = evaluator.evaluate_all_files()
evaluator.save_to_json(answers)

  0%|          | 0/107 [00:00<?, ?it/s]

Rate limit error: Rate limit reached for 10KTPM-200RPM in organization org-Y2tKyCPFO6tIjtCtOVZ7c9tr on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.. Retrying Question No. 5 Attempt:(1)


  0%|          | 0/107 [00:08<?, ?it/s]


KeyboardInterrupt detected. Saving results and closing.
Saved to: /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/labeled_text/../research_extractions/research_evaluations.json


AssertionError: 

04 - Convert results to a CSV

In [ ]:
import numpy as np

#Map responses to data
# Define a dictionary to map keywords in order or priority to values
keyword_mapping = {
    0: ["poor", "bad", "negative", "n", "no"],
    1: ["good", "excellent", "positive", "y", "yes"]
}


In [ ]:
json_path = r'/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/ocr/keyword_matching_labeled_text/text_evaluations/research_evaluations.json'

In [ ]:
from calvin_utils.gpt_sys_review.json_utils import CustomSummarizer

# Create an instance of CustomSummarizer
custom_summarizer = CustomSummarizer(json_path=json_path, keyword_mapping=keyword_mapping)

# Run the custom summarization and save the results to CSV
result_df = custom_summarizer.run_custom()

# Display the resulting DataFrame
print(result_df)